In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
def function(*, input: np.ndarray, theta: np.ndarray) -> np.ndarray:
    ones = np.ones(len(input)).reshape(-1, 1)
    x_add = np.hstack((ones, input))
    y = np.sum(x_add * theta.reshape(-1,), axis= 1)
    return y

def plot_diagram(*, name_1: str, name_2: str, mode_1: str, mode_2: str,
                  x_1: np.array, y_1: np.array, x_2: np.array, y_2: np.array):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_1,  y=y_1, mode= mode_1, name= name_1))
    fig.add_trace(go.Scatter(x=x_2,  y=y_2, mode= mode_2, name= name_2))
    fig.show()

def normalized(vector: np.ndarray) -> np.ndarray:
    mean = np.mean(vector)  # Mean of input vector
    standard_deviation = np.std(vector) # std of input vector
    normalized_vector = (vector - mean) / standard_deviation # linear transform vector by using gauss
    return normalized_vector

def convert_to_new_order(*, input: np.ndarray, order: int) -> np.ndarray:
    """
        Convert vector x into matrix x with multiple order
    """
    input = input.reshape(-1,)
    ones = np.ones((len(input), order))
    for i in range(0, order):
        ones[:, i] = input**(i+1)
    return ones

def true_value_theta(*, input: np.ndarray, output: np.ndarray) -> np.ndarray:
    """
        Compute true value of theta 
        theta =   (X.T*X)^-1 * (X.T * Y)
    """
    ones = np.ones(len(input)).reshape(-1, 1)
    x_add = np.hstack((ones, input))
    inv_XT_X = np.linalg.inv(np.dot(x_add.T, x_add))
    XT_Y = np.dot(x_add.T, output.reshape(-1, 1))
    theta_true = np.dot(inv_XT_X, XT_Y)
    return theta_true

def convert_theta_nomr_to_theta(*, theta_normalized: np.ndarray, input: np.ndarray, output: np.ndarray) -> np.ndarray:
    theta = np.zeros_like(theta_normalized)

    mean_x = np.mean(input,  axis= 0)
    std_x = np.std(input,  axis= 0)

    mean_y = np.mean(output, axis= 0)
    std_y = np.std(output, axis= 0)

    theta[1:] = std_y*theta_normalized[1:]/(std_x[1:].reshape(-1, 1))
    theta[0] = mean_y + std_y*theta_normalized[0] - np.dot(std_y*mean_x[1:]/std_x[1:], theta_normalized[1:])

    return theta

In [3]:
class Linear_Regression_Multivariables:
    def __init__(self, *, number_of_feature: int) -> None:
        self.number_of_features = number_of_feature

    def compute_normalized_vector(self, vector: np.ndarray) -> np.ndarray:
        mean = np.mean(vector)  # Mean of input vector
        standard_deviation = np.std(vector) # std of input vector
        normalized_vector = (vector - mean) / standard_deviation # linear transform vector by using gauss
        return normalized_vector
    
    def apply_normalize_matrix(self, *, input: np.ndarray, output: np.ndarray) -> np.ndarray:

        normalized_input = np.apply_along_axis(func1d= self.compute_normalized_vector, arr= input, axis= 0)
        normalized_input = normalized_input.reshape(-1, self.number_of_features)

        normalized_output = np.apply_along_axis(func1d= self.compute_normalized_vector, arr= output, axis= 0)
        normalized_output = normalized_output.reshape(-1, 1)
        return normalized_input, normalized_output

    def add_ones_columns(self, *, normalized_input: np.ndarray) -> np.ndarray:
        ones = np.ones(len(normalized_input)).reshape(-1, 1)
        x_add = np.hstack((ones, normalized_input))
        return x_add

    def predict(self, *, theta: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        y_pred = np.matmul(normalized_input, theta)
        return y_pred
    
    def compute_loss_function(self, *, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        m = len(y_true)
        E = y_pred - y_true
        J = np.sum((E)**2)/ (2*m)
        return J
    
    def update_params(self, *, theta: np.ndarray, lr: float, y_pred: np.ndarray, 
                      y_true: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        m = len(y_true)
        E = y_pred - y_true
        dJ_dtheta = np.dot(normalized_input.T, E) / (m)
        theta_updated = theta - lr*dJ_dtheta
        return theta_updated
    
    def convert_theta_nomr_to_theta(self, *, theta_normalized: np.ndarray, input: np.ndarray, output: np.ndarray) -> np.ndarray:
        theta = np.zeros_like(theta_normalized)

        mean_x = np.mean(input,  axis= 0)
        std_x = np.std(input,  axis= 0)

        mean_y = np.mean(output, axis= 0)
        std_y = np.std(output, axis= 0)

        theta[1:] = std_y*theta_normalized[1:]/(std_x.reshape(-1, 1))
        theta[0] = mean_y + std_y*theta_normalized[0] - np.dot(std_y*mean_x/std_x, theta_normalized[1:])

        return theta
    
    def train(self, *, epoch: int, theta: np.ndarray, input: np.ndarray, 
              output: np.ndarray, lr: float) -> np.ndarray:
        
        normalized_input, normalized_ouput = self.apply_normalize_matrix(input= input, output= output)
        normalized_input_add_ones = self.add_ones_columns(normalized_input= normalized_input)
        
        J_array = np.array([])
        for i in range(epoch):
            y_pred = self.predict(theta= theta, 
                                  normalized_input= normalized_input_add_ones)
            J = self.compute_loss_function(y_true= normalized_ouput, 
                                           y_pred= y_pred)
            theta = self.update_params(theta= theta, lr= lr, y_pred= y_pred, 
                                       y_true= normalized_ouput, normalized_input= normalized_input_add_ones)
            J_array = np.append(arr= J_array, values= J)
        
        theta = self.convert_theta_nomr_to_theta(theta_normalized= theta, input= input, output= output)

        return J_array, theta



### ex2

In [4]:
# Read csv file ex2.csv
pd_ex2 = pd.read_csv('ex2.csv')

# Get collumns of file 
X_cols = pd_ex2.columns[:-1]
Y_col = pd_ex2.columns[-1]

In [5]:
# Get vector input and output
X = pd_ex2[X_cols].values
Y = pd_ex2[Y_col].values

In [6]:
theta_init = np.random.randn(len(X_cols) + 1, 1)
theta_init.shape

(9, 1)

In [7]:
### 2. Training 
lrm = Linear_Regression_Multivariables(number_of_feature= 8)
J_array_ex_2, theta_ex_2 = lrm.train(
    epoch= 500000, theta= theta_init, input= X,
    output= Y, lr= 0.001
)

In [8]:
theta_real_ = true_value_theta(input= X, output= Y)
X_add = lrm.add_ones_columns(normalized_input= X)

In [9]:
theta_real_.shape

(9, 1)

In [10]:
# plot_diagram(name_1= 'a', name_2= 'b', mode_1= 'markers', mode_2= 'markers',
#              x_1= X.reshape(-1, ), y_1= Y.reshape(-1, ), 
#              x_2= X.reshape(-1, ), y_2= np.dot(X_add, theta_ex_2).reshape(-1, ))

In [11]:
np.round((theta_real_ - theta_ex_2) / theta_real_, 2)

array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [-0.  ],
       [-0.01],
       [ 0.  ]])

### ex3

In [12]:
# Read csv file ex2.csv
pd_ex3 = pd.read_csv('ex3.csv')

# Get collumns of file 
X_cols = pd_ex3.columns[:-1]
Y_col = pd_ex3.columns[-1]

In [13]:
# Get vector input and output
x_value = pd_ex3[X_cols].values
y_value = pd_ex3[Y_col].values

In [14]:
order= 4

X = convert_to_new_order(input= x_value, order= order)
Y = y_value.reshape(-1, 1)

In [15]:
theta_init = np.random.randn(order + 1, 1)
theta_init.shape

(5, 1)

In [16]:
### 2. Training 
lrm = Linear_Regression_Multivariables(number_of_feature= order)
J_array_ex_3, theta_ex_3 = lrm.train(
    epoch= 100000, theta= theta_init, input= X,
    output= Y, lr= 0.01
)

In [17]:
plot_diagram(name_1= 'a', name_2= 'b', mode_1= 'markers', mode_2= 'lines',
             x_1= x_value.reshape(-1, ), y_1= y_value.reshape(-1, ), 
             x_2= x_value.reshape(-1, ), y_2= function(input= X, theta= theta_ex_3))

In [18]:
theta_true = true_value_theta(input= X, output= Y)

In [19]:
plot_diagram(name_1= 'a', name_2= 'b', mode_1= 'markers', mode_2= 'lines',
             x_1= x_value.reshape(-1, ), y_1= y_value.reshape(-1, ), 
             x_2= x_value.reshape(-1, ), y_2= function(input= X, theta= theta_true))